In [1]:
import math
import numpy as np
import random
from time import time

import pandas as pd
import pandasql as ps
from downcast import reduce

import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression, load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb

import optuna

from model import Autoencoder, Regressor
import torch
from torch import nn, optim
from utils import *

# PREPARE FEATURE SETS

In [2]:
IOWA_PATH = '../../datasets/train_data_iowa.csv'
CF_PATH = "../../datasets/crafted_features.csv"

In [3]:
features = [
    "location",
    "location_x", "location_y",
    "restaurant_location_x", "restaurant_location_y", 
    "order_time",
    "etd",
    "restaurant_queue",
    "max_pre_shift",
    "max_post_shift",
    "restaurants_before_customer",
    "customers_before_customer",
    "len_vehicle_route_to_customer",
]

for i in range(23):
    features.append(f"vehicle_route_to_customer_pos_x_{i}")
    features.append(f"vehicle_route_to_customer_pos_y_{i}")
    features.append(f"vehicle_route_to_customer_action_{i}")
    features.append(f"vehicle_route_to_customer_time_action_{i}") 

In [4]:
# Import data
start_time = time()

usecols=[*features, "atd"]

meta = pd.read_csv(IOWA_PATH, header=0, sep=";", usecols=usecols, nrows=10)
meta = reduce(meta)
dtypes = dict(meta.dtypes)

raw = pd.read_csv(IOWA_PATH, header=0, sep=";", usecols=usecols, dtype = dtypes)

print(f"Elapsed time: {time() - start_time} seconds")
print(raw.info(verbose=False, memory_usage="deep"))
X = raw.loc[:, raw.columns != 'atd']
y = raw['atd'] - raw['etd']

pd.set_option("display.max_columns", len(raw.columns))
raw

Elapsed time: 65.22364139556885 seconds
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850469 entries, 0 to 850468
Columns: 106 entries, location to vehicle_route_to_customer_time_action_22
dtypes: float16(50), int16(4), int8(52)
memory usage: 129.8 MB
None


,location,order_time,atd,etd,restaurant_queue,max_pre_shift,max_post_shift,location_x,location_y,restaurant_location_x,restaurant_location_y,restaurants_before_customer,customers_before_customer,len_vehicle_route_to_customer,vehicle_route_to_customer_pos_x_0,vehicle_route_to_customer_pos_y_0,vehicle_route_to_customer_action_0,vehicle_route_to_customer_time_action_0,vehicle_route_to_customer_pos_x_1,vehicle_route_to_customer_pos_y_1,vehicle_route_to_customer_action_1,vehicle_route_to_customer_time_action_1,vehicle_route_to_customer_pos_x_2,vehicle_route_to_customer_pos_y_2,vehicle_route_to_customer_action_2,vehicle_route_to_customer_time_action_2,vehicle_route_to_customer_pos_x_3,vehicle_route_to_customer_pos_y_3,vehicle_route_to_customer_action_3,vehicle_route_to_customer_time_action_3,vehicle_route_to_customer_pos_x_4,vehicle_route_to_customer_pos_y_4,vehicle_route_to_customer_action_4,vehicle_route_to_customer_time_action_4,vehicle_route_to_customer_pos_x_5,vehicle_route_to_customer_pos_y_5,vehicle_route_to_customer_action_5,vehicle_route_to_customer_time_action_5,vehicle_route_to_customer_pos_x_6,vehicle_route_to_customer_pos_y_6,vehicle_route_to_customer_action_6,vehicle_route_to_customer_time_action_6,vehicle_route_to_customer_pos_x_7,vehicle_route_to_customer_pos_y_7,vehicle_route_to_customer_action_7,vehicle_route_to_customer_time_action_7,vehicle_route_to_customer_pos_x_8,vehicle_route_to_customer_pos_y_8,vehicle_route_to_customer_action_8,vehicle_route_to_customer_time_action_8,vehicle_route_to_customer_pos_x_9,vehicle_route_to_customer_pos_y_9,vehicle_route_to_customer_action_9,vehicle_route_to_customer_time_action_9,vehicle_route_to_customer_pos_x_10,vehicle_route_to_customer_pos_y_10,vehicle_route_to_customer_action_10,vehicle_route_to_customer_time_action_10,vehicle_route_to_customer_pos_x_11,vehicle_route_to_customer_pos_y_11,vehicle_route_to_customer_action_11,vehicle_route_to_customer_time_action_11,vehicle_route_to_customer_pos_x_12,vehicle_route_to_customer_pos_y_12,vehicle_route_to_customer_action_12,vehicle_route_to_customer_time_action_12,vehicle_route_to_customer_pos_x_13,vehicle_route_to_customer_pos_y_13,vehicle_route_to_customer_action_13,vehicle_route_to_customer_time_action_13,vehicle_route_to_customer_pos_x_14,vehicle_route_to_customer_pos_y_14,vehicle_route_to_customer_action_14,vehicle_route_to_customer_time_action_14,vehicle_route_to_customer_pos_x_15,vehicle_route_to_customer_pos_y_15,vehicle_route_to_customer_action_15,vehicle_route_to_customer_time_action_15,vehicle_route_to_customer_pos_x_16,vehicle_route_to_customer_pos_y_16,vehicle_route_to_customer_action_16,vehicle_route_to_customer_time_action_16,vehicle_route_to_customer_pos_x_17,vehicle_route_to_customer_pos_y_17,vehicle_route_to_customer_action_17,vehicle_route_to_customer_time_action_17,vehicle_route_to_customer_pos_x_18,vehicle_route_to_customer_pos_y_18,vehicle_route_to_customer_action_18,vehicle_route_to_customer_time_action_18,vehicle_route_to_customer_pos_x_19,vehicle_route_to_customer_pos_y_19,vehicle_route_to_customer_action_19,vehicle_route_to_customer_time_action_19,vehicle_route_to_customer_pos_x_20,vehicle_route_to_customer_pos_y_20,vehicle_route_to_customer_action_20,vehicle_route_to_customer_time_action_20,vehicle_route_to_customer_pos_x_21,vehicle_route_to_customer_pos_y_21,vehicle_route_to_customer_action_21,vehicle_route_to_customer_time_action_21,vehicle_route_to_customer_pos_x_22,vehicle_route_to_customer_pos_y_22,vehicle_route_to_customer_action_22,vehicle_route_to_customer_time_action_22
0,2097,567,583,584,8,0,0,4.562500,8.312500,4.996094,10.343750,1,1,4,4.996094,10.343750,1,4,4.996094,10.343750,3,5,4.562500,8.312500,2,5,4.562500,8.312500,4,3,0.000000,0.000000,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0
1,997,58

# Feature Engineering

Features used in Hildebrandt et al. (2020):
<ul>
    <li>n_stops: sum(vehicle_route_to_customer_action_i = 1 or 2)</li>
    <li>n_pickup_stops: sum(vehicle_route_to_customer_action_i = 1)</li>
    <li>n_delivery_stops: sum(vehicle_route_to_customer_action_i = 2)</li>
    <li>max_pre_shift: already given</li>    
    <li>max_post_shift: already given</li>
    <li>prep_time: already given ( == restaurant_queue)</li>
    <li>order_time: already given</li>
    <li>eta_pom: already given</li>
    <li>customer_location: already given</li>
    <li>restaurant_location: already given</li>
</ul>

Weitere:
<ul>
    <li>restaurants_before_customer : already given</li>
    <li>customers_before_customer : already given</li>
    <li>dropoff_time</li>

In [ ]:
#Define strings to identify needed columns for each feature we want to craft
query_strings = {
    #"n_stops" : ["vehicle_route_to_customer_action"],
        
    #"n_pickup_stops" : ["vehicle_route_to_customer_action"],
    
    #"n_delivery_stops" : ["vehicle_route_to_customer_action"],
    "dropoff_time" : ["vehicle_route_to_customer_action",
                      "vehicle_route_to_customer_time_action",
                      "vehicle_route_to_customer_pos_x_",
                      "vehicle_route_to_customer_pos_y_",
                      "location_x", "location_y"
                     ]
    #"cumulated_prep_time" : 
    #"cumulated_dropoff_time" :
    #"cumulated_delivery_time" : 
}

raw_feats = [
    "location_x", "location_y",
    "restaurant_location_x", "restaurant_location_y",
    "etd", 
    "atd", 
    "order_time", 
    "max_pre_shift", 
    "max_post_shift", 
    "restaurant_queue",
    "restaurants_before_customer", 
    "customers_before_customer",
    "len_vehicle_route_to_customer",
    
    
]

mask = pd.DataFrame()
feats = pd.DataFrame()

# First, add used raw features to feats
for feat in raw_feats:
    feats[feat] = raw[feat]

# Craft features and add to feats
for key,value in query_strings.items():
    
    needed_columns = [col for col in raw.columns if any(x in col for x in value)]
    inp = raw[needed_columns]
    
    if key == "n_stops":
        for col in inp:
            mask[col] = (inp[col] > 0) & (inp[col] < 3)
            feats[key] = mask.sum(axis=1)
    
    if key == "n_pickup_stops": 
        for col in inp:
            mask[col] = inp[col] == 1
            feats[key] = mask.sum(axis=1)
    
    if key == "n_delivery_stops": 
        for col in inp:
            mask[col] = inp[col] == 2
            feats[key] = mask.sum(axis=1)
    
    if key == "dropoff_time" :
        inp_copy = inp.drop(["restaurant_location_x", "restaurant_location_y"], axis=1)
        actions = inp_copy[[f for f in inp_copy if "vehicle_route_to_customer_action" in f]] 
        
        dropoff_times = []
        
        for index, row in actions.head(50).iterrows():
            dropoff_action = []
            for i, v in row.items():
                if v == 4:
                    dropoff_action.append(i)
            customer_dropoff = dropoff_action[-1]
            print(index)
            dropoff_times.append(
                inp_copy.at[index, f"vehicle_route_to_customer_time_action_{customer_dropoff[-1]}"]
            )
        dropoff_np = np.asarray(dropoff_times)

In [ ]:
feats

In [ ]:
feats.to_csv(CF_PATH, sep=";")

In [ ]:
crafted = pd.read_csv(CF_PATH, sep=";", index_col=[0])
crafted

# Data description

In [ ]:
sns.displot(crafted["order_time"], 
            kind="kde",
            bw_adjust=1,
            height=4, aspect=6/4,
            legend=True).savefig("Plots/order_time_dist.png")

In [ ]:
sns.displot(crafted["atd"]-crafted["etd"], 
            kind="kde", 
            bw_adjust=2,
            height=4, aspect=6/4).savefig("Plots/delivery_delay.png")

In [ ]:
sns.displot(
    crafted["restaurant_queue"],
    kind="kde",
    bw_adjust = 2,
    height = 4, aspect=6/4
).savefig("Plots/prep_time.png")

In [ ]:
customer_locations = np.asarray(list(set(zip(raw.location_x, raw.location_y))))
customer_locations_x = [t[0] for t in customer_locations]
customer_locations_y = [t[1] for t in customer_locations]

restaurant_locations = list(set(zip(raw.restaurant_location_x, raw.restaurant_location_y)))
restaurant_locations_x = [t[0] for t in restaurant_locations]
restaurant_locations_y = [t[1] for t in restaurant_locations]

print(customer_locations.shape)
plt.scatter(customer_locations_x, customer_locations_y, s=0.1)
plt.scatter(restaurant_locations_x, restaurant_locations_y, s=10, marker="h")
plt.xlabel("latitude")
plt.ylabel("longitude")
plt.savefig("Plots/spatial_dist")

plt.show()

# Train() of NN, temporary in this notebook

In [5]:
def train(model, data, feature_list, params):
    
    torch.manual_seed(0)
    np.random.seed(0)
    random.seed(0)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print("Importing data.")

    etd_dataset = ETDData(data=data, feature_list=feature_list, objective=model.name)
    split = DataSplit(etd_dataset, shuffle=True)
    trainloader, _, testloader = split.get_split(batch_size=params["batch_size"], num_workers=8)
    
    print("Start training.")
    patience = params["patience"]
    criterion = params["criterion"]  # define your loss function and optimizer
    optimizer = params["optimizer"]

    
    early_stopping = EarlyStopping(patience=params["patience"], verbose=True) 
    epochs = params["epochs"] # How many epochs do you want to train?
    
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            # get the inputs; data is a list of [inputs, labels]
            inputs = inputs.float().to(device)
            labels = labels.float().view(-1, model.view).to(device) 
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels) 
            loss.backward()
            optimizer.step()
            # print statistics
            running_loss += loss.item()
        test_loss = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs = inputs.float().to(device)
                labels = labels.float().view(-1, model.view).to(device) 
                logps = model.forward(inputs)
                batch_loss = criterion(logps, labels)
                test_loss += batch_loss.item()
        print(f"Epoch {epoch+1}/{epochs}.. "
                f"Train loss: {running_loss / len(trainloader):.3f}.. "
                f"Test loss: {test_loss / len(testloader):.3f}.. ")
        early_stopping(test_loss / len(testloader), model)
        if early_stopping.early_stop:
            print("Early stopping")
            break
        model.train()
        
    print('Finished Training')
    #model.load_state_dict(torch.load('checkpoint.pt'))
    #model.save(ae, 'perceptron.pth')
    return model, abs(early_stopping.best_score)

# Study

## Part 1: Different sample sizes 

With the first part, we seek to examine the convergence behavior of our models and answer following question: How many samples are enough to train the model without ? 
We determine the answer to that question graphically. For that, we construct plots where the x-axis represents the number of samples used in the corresponding training instance, and the y-axis represents the corresponding L<sub>2</sub>-loss measured with the mean squared error.

### Define helper functions ###

In [ ]:
def plot_convergence(sample_sizes, results, title):
    plt.xlabel("Sample size")
    plt.ylabel("Mean squared error")
    plt.plot(sample_sizes, results)
    plt.savefig(f"Plots/{title}.png")

# Helper function for lightgbm
def best_iteration(evals_result):
    iterations = evals_result
    small = iterations[0]
    for i in iterations:
        if small > i:
            small = i
     
    return small    

### Test 1.1: Tree-based ensembles: GBDT and RF (LightGBM Implementation)

In [ ]:
### Convergence Test for LightGBM's GBDT ###
def lightgbm_sample_size_testing(X, y, params, start=1000, stop=200000, step=1000, early_stopping_rounds=10):
    
    sample_sizes = np.arange(start=start, stop=stop, step=step)
    results = []
    
    evals_result = {}
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=42)
    
    for rows in sample_sizes:
        train_set = lgb.Dataset(X_train[:rows],y_train[:rows])
        val_set = lgb.Dataset(X_test, y_test, reference=train_set)
        bst = lgb.train(
            params,
            train_set=train_set,
            valid_sets=[val_set, train_set],
            evals_result = evals_result,
        )
        best_iter = best_iteration(evals_result=dict(evals_result["valid_0"])["l2"])
        results.append(best_iter)
    plot_convergence(sample_sizes, results, "gbdt_convergence")

In [ ]:
params_gbdt = {
    "boosting_type" : "gbdt",
    "metrics" : "l2",
    "learning_rate" : 0.02, 
    "num_threads"  : 6,
    "random_state" : 42,
    "force_row_wise" : True,
    "n_estimators" : 500,
    "early_stopping_rounds" : 20, 
}

lightgbm_sample_size_testing(X, y, params_gbdt, 1000, 101000, 1000)

In [ ]:
params_rf = {
    "boosting_type" : "rf",
    "metrics" : "l2", 
    "num_leaves" : 500,
    "learning_rate" : 0.0001,
    "n_estimators" : 500,
    "bagging_fraction" : 0.632,
    "bagging_freq" : 1,
    "feature_fraction" : 0.632,
    "min_data_in_leaf" : 1,
    "min_data_in_bin" : 1,
    "num_threads"  : 6,
    "random_state" : 42,
    "force_row_wise" : True,
    "early_stopping_rounds" : 20
}

lightgbm_sample_size_testing(X, y, params_gbdt, 1000, 101000, 1000)

### Test 1.2: Linear Regression

In [ ]:
### Convergence test for Scikit-Learn's Linear Regression ###
def lr_sample_size_testing(X, y, params=None, start=1000, stop=100000, step=1000):
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=42)
    sample_sizes = np.arange(start=start, stop=stop, step=step)
    results = []
    
    for rows in sample_sizes:
        lr = LinearRegression()
        lr.fit(X_train[:rows], y_train[:rows])
        mse = mean_squared_error(y_test, lr.predict(X_test))
        print(f"Sample size - error : {rows} - {mse}")
        results.append(mse)
    plot_convergence(sample_sizes, results, "LR_Convergence")

lr_sample_size_testing(X, y)

### Test 1.3: Single Layer Perceptron

In [ ]:
sample_sizes = [1000,10000,100000]
results = []

n_features = len(features)
n_hidden = math.ceil(n_features * (1 / 2))
n_code = math.ceil(n_hidden * (1 / 2))

ae = Autoencoder(n_features=n_features, n_hidden=n_hidden, n_code=n_code)
slp = Regressor(n_features = n_features, n_hidden = n_hidden, n_output = 1)

params = {
        "patience" : 10,
        "criterion" : nn.MSELoss(),
        "optimizer" : optim.Adam(slp.parameters(), lr=0.0001),
        "epochs" : 100,
        "batch_size" : 50,
}

for rows in sample_sizes:
    print(f"Sample size {rows}")
    model, mse = train(slp, raw[:rows], features, params)
    results.append(mse)
    print(f"Mean squared error: {mse} for sample size: {rows}")
plot_convergence(sample_sizes, results, "nn_convergence")

## Part 2: Hyperparameter optimization

In [14]:
def gbdt_opt(trial, X, y, boosting_type, sample_size=100000):
    
    sample_size=sample_size
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=42)
    train_set = lgb.Dataset(X_train[:sample_size],y_train[:sample_size])
    valid_set = lgb.Dataset(X_test, y_test)
    
    params = {
        "gbdt" : {
            "boosting_type" : "gbdt",
            "metric" : "l2",
            "objective" : "regression",
            "learning_rate" : trial.suggest_uniform("learning_rate", 0.01, 0.05),
            "num_leaves" : trial.suggest_int("num_leaves", 30, 200),
            "min_data_in_leaf" : trial.suggest_int("min_child_samples", 1, 50),
            "max_bin" : 1000,
            "num_threads"  : 6,
            "random_state" : 42,
            "force_row_wise" : True, 
            "num_boost_round": 1000,
            "early_stopping" : 20
            
        },
        "rf" : {
            
        }      
    }
    evals_result = {}
    bst = lgb.train(
        params[boosting_type],
        train_set=train_set,
        valid_sets=[valid_set, train_set],
        valid_names=["Validation error", "Train error"],
        evals_result = evals_result,
        verbose_eval = 5,
    )
    preds = bst.predict(X_test)
    loss  = mean_squared_error(y_test, preds)
    return loss

In [15]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.CmaEsSampler(seed=42))
study.optimize(lambda trial: gbdt_opt(trial, X, y, "gbdt", 100000), n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("Value: {}".format(trial.value))
print("Params: ")
for key, value in trial.params.items():
    print("{}: {}".format(key, value))   

[I 2021-02-02 20:51:05,324] A new study created in memory with name: no-name-2e5e7b7a-2942-4d28-aae1-65403125cbce


[LightGBM] [Info] Total Bins 37583
[LightGBM] [Info] Number of data points in the train set: 100000, number of used features: 93
[LightGBM] [Info] Start training from score 4.189170
Training until validation scores don't improve for 20 rounds
[5]	Train error's l2: 38.133	Validation error's l2: 37.9376
[10]	Train error's l2: 35.8657	Validation error's l2: 35.815
[15]	Train error's l2: 34.0525	Validation error's l2: 34.1232
[20]	Train error's l2: 32.6034	Validation error's l2: 32.7979
[25]	Train error's l2: 31.4149	Validation error's l2: 31.725
[30]	Train error's l2: 30.4349	Validation error's l2: 30.8601
[35]	Train error's l2: 29.6171	Validation error's l2: 30.1638
[40]	Train error's l2: 28.9329	Validation error's l2: 29.5972
[45]	Train error's l2: 28.3489	Validation error's l2: 29.1318
[50]	Train error's l2: 27.8591	Validation error's l2: 28.7565
[55]	Train error's l2: 27.4294	Validation error's l2: 28.436
[60]	Train error's l2: 27.0535	Validation error's l2: 28.1758
[65]	Train error's

[I 2021-02-02 20:51:20,872] Trial 0 finished with value: 26.164621055383076 and parameters: {'learning_rate': 0.0249816047538945, 'num_leaves': 122, 'min_child_samples': 15}. Best is trial 0 with value: 26.164621055383076.
/home/emre/Bachelor/venv/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning:

Found `num_boost_round` in params. Will use it instead of argument

/home/emre/Bachelor/venv/lib/python3.8/site-packages/lightgbm/engine.py:156: UserWarning:

Found `early_stopping` in params. Will use it instead of argument



[LightGBM] [Info] Total Bins 37583
[LightGBM] [Info] Number of data points in the train set: 100000, number of used features: 93
[LightGBM] [Info] Start training from score 4.189170
Training until validation scores don't improve for 20 rounds
[5]	Train error's l2: 37.7721	Validation error's l2: 37.5943
[10]	Train error's l2: 35.3169	Validation error's l2: 35.286
[15]	Train error's l2: 33.4098	Validation error's l2: 33.5101
[20]	Train error's l2: 31.9327	Validation error's l2: 32.1607
[25]	Train error's l2: 30.7536	Validation error's l2: 31.1073
[30]	Train error's l2: 29.7897	Validation error's l2: 30.2681
[35]	Train error's l2: 29.004	Validation error's l2: 29.6095
[40]	Train error's l2: 28.3598	Validation error's l2: 29.0887
[45]	Train error's l2: 27.8289	Validation error's l2: 28.6789
[50]	Train error's l2: 27.3675	Validation error's l2: 28.3355
[55]	Train error's l2: 26.9756	Validation error's l2: 28.0599
[60]	Train error's l2: 26.6214	Validation error's l2: 27.8374
[65]	Train error

[I 2021-02-02 20:51:34,314] Trial 1 finished with value: 26.14814418980261 and parameters: {'learning_rate': 0.02874843656877058, 'num_leaves': 115, 'min_child_samples': 25}. Best is trial 1 with value: 26.14814418980261.
/home/emre/Bachelor/venv/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning:

Found `num_boost_round` in params. Will use it instead of argument

/home/emre/Bachelor/venv/lib/python3.8/site-packages/lightgbm/engine.py:156: UserWarning:

Found `early_stopping` in params. Will use it instead of argument



...

[640]	Train error's l2: 25.6606	Validation error's l2: 26.7725
[645]	Train error's l2: 25.6431	Validation error's l2: 26.7634
[650]	Train error's l2: 25.628	Validation error's l2: 26.7579
[655]	Train error's l2: 25.6135	Validation error's l2: 26.753
[660]	Train error's l2: 25.5969	Validation error's l2: 26.7438
[665]	Train error's l2: 25.5815	Validation error's l2: 26.737
[670]	Train error's l2: 25.565	Validation error's l2: 26.728
[675]	Train error's l2: 25.5513	Validation error's l2: 26.724
[680]	Train error's l2: 25.537	Validation error's l2: 26.7177
[685]	Train error's l2: 25.5215	Validation error's l2: 26.71
[690]	Train error's l2: 25.5056	Validation error's l2: 26.7034
[695]	Train error's l2: 25.4898	Validation error's l2: 26.6951
[700]	Train error's l2: 25.4762	Validation error's l2: 26.6898
[705]	Train error's l2: 25.4613	Validation error's l2: 26.6846
[710]	Train error's l2: 25.4471	Validation error's l2: 26.6783
[715]	Train error's l2: 25.4334	Validation error's l2: 26.6721
[

[I 2021-02-02 21:20:27,450] Trial 98 finished with value: 26.471672982398687 and parameters: {'learning_rate': 0.01, 'num_leaves': 30, 'min_child_samples': 50}. Best is trial 18 with value: 26.1044868897269.
/home/emre/Bachelor/venv/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning:

Found `num_boost_round` in params. Will use it instead of argument

/home/emre/Bachelor/venv/lib/python3.8/site-packages/lightgbm/engine.py:156: UserWarning:

Found `early_stopping` in params. Will use it instead of argument



[LightGBM] [Info] Total Bins 37608
[LightGBM] [Info] Number of data points in the train set: 100000, number of used features: 101
[LightGBM] [Info] Start training from score 4.189170
Training until validation scores don't improve for 20 rounds
[5]	Train error's l2: 35.8034	Validation error's l2: 35.8378
[10]	Train error's l2: 32.5279	Validation error's l2: 32.8732
[15]	Train error's l2: 30.3371	Validation error's l2: 30.9532
[20]	Train error's l2: 28.8197	Validation error's l2: 29.6975
[25]	Train error's l2: 27.7428	Validation error's l2: 28.8654
[30]	Train error's l2: 26.936	Validation error's l2: 28.2949
[35]	Train error's l2: 26.3012	Validation error's l2: 27.9028
[40]	Train error's l2: 25.7696	Validation error's l2: 27.596
[45]	Train error's l2: 25.3023	Validation error's l2: 27.3434
[50]	Train error's l2: 24.907	Validation error's l2: 27.1685
[55]	Train error's l2: 24.5625	Validation error's l2: 27.0321
[60]	Train error's l2: 24.2136	Validation error's l2: 26.8792
[65]	Train error

[I 2021-02-02 21:20:34,851] Trial 99 finished with value: 26.280140967034555 and parameters: {'learning_rate': 0.0499999999, 'num_leaves': 110, 'min_child_samples': 1}. Best is trial 18 with value: 26.1044868897269.


Number of finished trials: 100
Best trial:
Value: 26.1044868897269
Params: 
learning_rate: 0.01
num_leaves: 113
min_child_samples: 50


In [16]:
optuna.visualization.plot_param_importances(study)

# Part 3: Introducing Noise

# NEURAL NETWORK (Pytorch)

# 1. Model definition

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Autoencoder(torch.nn.Module):
    def __init__(self, n_features, n_hidden, n_code):
        super(Autoencoder, self).__init__()
        self.name = "ae"
        self.view = n_features
        self.hidden_enc = nn.Linear(n_features, n_hidden)
        self.encode = nn.Linear(n_hidden, n_code)
        self.hidden_dec = nn.Linear(n_code, n_hidden)
        self.decode = nn.Linear(n_hidden, n_features)

    def forward(self, x):
        x = F.leaky_relu(self.hidden_enc(x))
        x = F.leaky_relu(self.encode(x))
        x = F.leaky_relu(self.hidden_dec(x))
        x = self.decode(x)

        return x
    
class Regressor(torch.nn.Module):
    def __init__(self, n_features, n_hidden, n_output):
        super(Regressor, self).__init__()
        self.name = "regressor"
        self.view = n_output
        self.hidden = nn.Linear(n_features, n_hidden)
        self.predict = nn.Linear(n_hidden, n_output)
    
    def forward(self, x):
        x = F.leaky_relu(self.hidden(x))
        x = self.predict(x)

        return x

## 2. Training loop

In [ ]:
import torch
from torch import nn, optim
from utils import *
    

def train(model, data, feature_list, fit_params):
    
    torch.manual_seed(0)
    np.random.seed(0)
    random.seed(0)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print("Importing data.")

    etd_dataset = ETDData(data=data, feature_list=feature_list, objective=model.name)
    split = DataSplit(etd_dataset, shuffle=True)
    trainloader, _, testloader = split.get_split(batch_size=params["batch_size"], num_workers=8)
    
    print("Start training.")
    patience = params["patience"]
    criterion = params["criterion"]  # define your loss function and optimizer
    optimizer = params["optimizer"]

    
    early_stopping = EarlyStopping(patience=params["patience"], verbose=True) 
    epochs = params["epochs"] # How many epochs do you want to train?
    
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            # get the inputs; data is a list of [inputs, labels]
            inputs = inputs.float().to(device)
            labels = labels.float().view(-1, model.view).to(device) 
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels) 
            loss.backward()
            optimizer.step()
            # print statistics
            running_loss += loss.item()
        test_loss = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs = inputs.float().to(device)
                labels = labels.float().view(-1, model.view).to(device) 
                logps = model.forward(inputs)
                batch_loss = criterion(logps, labels)
                test_loss += batch_loss.item()
        print(f"Epoch {epoch+1}/{epochs}.. "
                f"Train loss: {running_loss / len(trainloader):.3f}.. "
                f"Test loss: {test_loss / len(testloader):.3f}.. ")
        early_stopping(test_loss / len(testloader), model)
        if early_stopping.early_stop:
            print("Early stopping")
            break
        model.train()
        
    print('Finished Training')
    #ae.load_state_dict(torch.load('checkpoint.pt'))
    #torch.save(ae, 'perceptron.pth')
    return model, abs(early_stopping.best_score)

## Main